# Experiment: ATAC All-Sample Clustering

Objective:
- Build one ATAC object from all samples with ATAC fragments in `data-RNA`.
- Cluster all ATAC cells with SnapATAC2 and save checkpointed outputs.


In [1]:
from __future__ import annotations

import importlib
from pathlib import Path

import anndata as ad
import pandas as pd
import scanpy as sc
import snapatac2 as snap

import utils.dbit_rna_reader as dbit_rna_reader

dbit_rna_reader = importlib.reload(dbit_rna_reader)
discover_rna_tissue_pairs = dbit_rna_reader.discover_rna_tissue_pairs
discover_atac_fragment_tars = dbit_rna_reader.discover_atac_fragment_tars
extract_atac_fragment_archives = dbit_rna_reader.extract_atac_fragment_archives
import_atac_fragments_to_h5ad_per_sample = dbit_rna_reader.import_atac_fragments_to_h5ad_per_sample
write_rna_h5ad_per_sample = dbit_rna_reader.write_rna_h5ad_per_sample

sc.settings.verbosity = 2
sc.settings.set_figure_params(figsize=(5, 5), frameon=False)
print('snapatac2:', snap.__version__)


snapatac2: 2.8.0


/Users/christoffer/miniconda3/envs/cellcharter310/lib/python3.10/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


## Step 1 - Discover Samples and ATAC Tar Files


In [3]:
data_root = Path('data-RNA/atac')
pairs = discover_rna_tissue_pairs(data_root, recursive=True)
required = {'RNA_matrix', 'tissue_positions_list'}
sample_ids = sorted([s for s, files in pairs.items() if required.issubset(files)])

if not sample_ids:
    raise RuntimeError(f'No complete RNA+tissue sample pairs found under {data_root}')

atac_tar_manifest = discover_atac_fragment_tars(
    data_dir=data_root,
    sample_ids=sample_ids,
    recursive=True,
)

print('RNA sample count:', len(sample_ids))
print('ATAC tar sample count:', len(atac_tar_manifest))
atac_tar_manifest


RNA sample count: 7
ATAC tar sample count: 7


,sample_id,atac_kind,atac_tar
0,06_LPC5S1,atac_fragments,data-RNA/atac/06_LPC5S1_atac_fragments.tsv.tar
1,06_LPC5S2,atac_fragments,data-RNA/atac/06_LPC5S2_atac_fragments.tsv.tar
2,07_LPC10S1,atac_fragments,data-RNA/atac/07_LPC10S1_atac_fragments.tsv.tar
3,07_LPC10S2,atac_fragments,data-RNA/atac/07_LPC10S2_atac_fragments.tsv.tar
4,08_LPC21S1,atac_fragments,data-RNA/atac/08_LPC21S1_atac_fragments.tsv.tar
5,08_LPC21S2,atac_fragments,data-RNA/atac/08_LPC21S2_atac_fragments.tsv.tar
6,LPC5_Saggital,atac_fragments,data-RNA/atac/LPC5_Saggital_atac_fragments.tsv...


## Step 2 - Checkpoint RNA h5ad and Extract ATAC Fragments

RNA checkpoints are used to derive a barcode whitelist for ATAC import.


In [4]:
checkpoint_dir = Path('data/checkpoints')
rna_h5ad_dir = checkpoint_dir / 'rna_h5ad_by_sample'
atac_frag_dir = checkpoint_dir / 'atac_fragments_local'
atac_h5ad_dir = checkpoint_dir / 'atac_h5ad_by_sample'

rna_h5ad_manifest = write_rna_h5ad_per_sample(
    data_dir=data_root,
    out_dir=rna_h5ad_dir,
    sample_ids=sample_ids,
    overwrite=False,
    recursive=True,
)

dbit_atac_manifest = extract_atac_fragment_archives(
    out_dir=atac_frag_dir,
    atac_manifest=atac_tar_manifest,
    overwrite=False,
)

print(rna_h5ad_manifest[['sample_id', 'status', 'n_obs']])
print('Extracted ATAC fragment sets:', len(dbit_atac_manifest))
dbit_atac_manifest


       sample_id            status  n_obs
0      06_LPC5S1  skipped_existing   8736
1      06_LPC5S2  skipped_existing   8169
2     07_LPC10S1  skipped_existing   8180
3     07_LPC10S2  skipped_existing   8928
4     08_LPC21S1  skipped_existing   8485
5     08_LPC21S2  skipped_existing   8258
6  LPC5_Saggital  skipped_existing   9403
Extracted ATAC fragment sets: 7


,sample_id,atac_kind,atac_tar,fragments_tsv_gz,fragments_tbi
0,06_LPC5S1,atac_fragments,data-RNA/atac/06_LPC5S1_atac_fragments.tsv.tar,data/checkpoints/atac_fragments_local/06_LPC5S...,data/checkpoints/atac_fragments_local/06_LPC5S...
1,06_LPC5S2,atac_fragments,data-RNA/atac/06_LPC5S2_atac_fragments.tsv.tar,data/checkpoints/atac_fragments_local/06_LPC5S...,data/checkpoints/atac_fragments_local/06_LPC5S...
2,07_LPC10S1,atac_fragments,data-RNA/atac/07_LPC10S1_atac_fragments.tsv.tar,data/checkpoints/atac_fragments_local/07_LPC10...,data/checkpoints/atac_fragments_local/07_LPC10...
3,07_LPC10S2,atac_fragments,data-RNA/atac/07_LPC10S2_atac_fragments.tsv.tar,data/checkpoints/atac_fragments_local/07_LPC10...,data/checkpoints/atac_fragments_local/07_LPC10...
4,08_LPC21S1,atac_fragments,data-RNA/atac/08_LPC21S1_atac_fragments.tsv.tar,data/checkpoints/atac_fragments_local/08_LPC21...,data/checkpoints/atac_fragments_local/08_LPC21...
5,08_LPC21S2,atac_fragments,data-RNA/atac/08_LPC21S2_atac_fragments.tsv.tar,data/checkpoints/atac_fragments_local/08_LPC21...,data/checkpoints/atac_fragments_local/08_LPC21...
6,LPC5_Saggital,atac_fragments,data-RNA/atac/LPC5_Saggital_atac_fragments.tsv...,data/checkpoints/atac_fragments_local/LPC5_Sag...,data/checkpoints/atac_fragments_local/LPC5_Sag...


## Step 3 - Import ATAC Per Sample and Checkpoint as h5ad

This runs one sample at a time to avoid out-of-memory crashes.


In [ ]:
RUN_LOCAL_ATAC_IMPORT = True
USE_RNA_BARCODE_WHITELIST = True
CONTINUE_ON_ERROR = True
OVERWRITE_ATAC_H5AD = False
ATAC_MIN_FRAGMENTS = 1

if RUN_LOCAL_ATAC_IMPORT:
    # Set genome for your dataset, e.g. snap.genome.mm10 or snap.genome.hg38.
    genome = snap.genome.mm10

    whitelist_by_sample = None
    if USE_RNA_BARCODE_WHITELIST:
        whitelist_by_sample = {}
        for row in rna_h5ad_manifest.itertuples(index=False):
            sample_id = str(row.sample_id)
            a = sc.read_h5ad(str(row.rna_h5ad))
            barcodes = a.obs['barcode'].astype(str).tolist() if 'barcode' in a.obs.columns else a.obs_names.astype(str).tolist()
            whitelist_by_sample[sample_id] = barcodes
            del a

    atac_h5ad_manifest = import_atac_fragments_to_h5ad_per_sample(
        out_dir=atac_h5ad_dir,
        genome=genome,
        atac_manifest=dbit_atac_manifest,
        whitelist_by_sample=whitelist_by_sample,
        sorted_by_barcode=False,
        overwrite=OVERWRITE_ATAC_H5AD,
        continue_on_error=CONTINUE_ON_ERROR,
        build_tile_matrix=True,
        min_num_fragments=ATAC_MIN_FRAGMENTS,
    )

    print(atac_h5ad_manifest[['sample_id', 'status', 'n_obs', 'n_vars', 'error']])
    nonempty = int((atac_h5ad_manifest['n_obs'] > 0).sum())
    print(f'Non-empty ATAC sample h5ad files: {nonempty}/{len(atac_h5ad_manifest)}')
    atac_h5ad_manifest
else:
    print('Set RUN_LOCAL_ATAC_IMPORT=True to build ATAC checkpoints.')


       sample_id            status  n_obs   n_vars error
0      06_LPC5S1  skipped_existing   8736  5267565      
1      06_LPC5S2  skipped_existing   8169  5267565      
2     07_LPC10S1  skipped_existing   8180  5267565      
3     07_LPC10S2  skipped_existing   8928  5267565      
4     08_LPC21S1  skipped_existing   8485  5267565      
5     08_LPC21S2  skipped_existing   8258  5267565      
6  LPC5_Saggital  skipped_existing   9403  5267565      
Non-empty ATAC sample h5ad files: 7/7


: 

## Step 4 - Build One Combined ATAC Object


In [ ]:
if 'atac_h5ad_manifest' not in globals():
    atac_h5ad_manifest = pd.DataFrame(
        {
            'sample_id': p.name.replace('.atac.h5ad', ''),
            'atac_h5ad': str(p),
            'status': 'existing',
            'n_obs': -1,
            'n_vars': -1,
            'error': '',
        }
        for p in sorted((Path('data/checkpoints') / 'atac_h5ad_by_sample').glob('*.atac.h5ad'))
    )

valid = atac_h5ad_manifest.loc[(atac_h5ad_manifest['status'] != 'error') & (atac_h5ad_manifest['n_obs'] != 0)].copy()
if valid.empty:
    raise RuntimeError('No non-empty ATAC sample checkpoints found. Re-run Step 3.')

samples_to_build = sorted(valid['sample_id'].astype(str).tolist())
print('Building combined ATAC from samples:', samples_to_build)

atac_parts = []
for sample_id in samples_to_build:
    row = valid.loc[valid['sample_id'].astype(str) == sample_id].iloc[0]
    a = sc.read_h5ad(str(row['atac_h5ad']))
    if a.n_obs == 0:
        continue
    a.obs['sample_id'] = str(sample_id)
    a.obs['barcode'] = pd.Index(a.obs_names.astype(str)).str.replace(r'-[0-9]+$', '', regex=True).to_numpy()
    a.obs_names = (a.obs['sample_id'].astype(str) + ':' + a.obs['barcode'].astype(str)).to_numpy()
    a.obs_names_make_unique()
    atac_parts.append(a)

if not atac_parts:
    raise RuntimeError('No ATAC samples with cells remained after loading.')

atac = ad.concat(atac_parts, join='outer', merge='first', fill_value=0)
print(atac)
atac


Building combined ATAC from samples: ['06_LPC5S1', '06_LPC5S2', '07_LPC10S1', '07_LPC10S2', '08_LPC21S1', '08_LPC21S2', 'LPC5_Saggital']


## Step 5 - ATAC Preprocessing and Clustering


In [ ]:
if getattr(atac, 'X', None) is None:
    raise RuntimeError('ATAC matrix missing (atac.X is None). Re-run Step 3 with build_tile_matrix=True.')

snap.pp.select_features(atac)
snap.tl.spectral(atac)
snap.pp.knn(atac)
snap.tl.umap(atac)
snap.tl.leiden(atac, key_added='leiden_1')

print(atac)
atac


In [ ]:
snap.pl.umap(atac, color=['sample_id', 'leiden_1'], show=True)


## Step 6 - Save Outputs


In [ ]:
out_dir = Path('data/results')
out_dir.mkdir(parents=True, exist_ok=True)

atac.write_h5ad(out_dir / 'atac_all_samples_clustered.h5ad')
print('Saved:', out_dir / 'atac_all_samples_clustered.h5ad')
